In [ ]:
import pandas as pd
import numpy as np
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
import datetime as dt
import statistics as st
import time as timeee
import statistics

In [ ]:
depression_words = ['depressao', 'depressão', 'depressiva', 'depressivo',
    'deprimido', 'deprimida', 'suicídio', 'suicidio', 'suicidar',
    'triste', 'tristeza', 'infeliz', 'ansiedade', 'ansioso', 
    'ansiosa', 'morte', 'mortes','morrer', 'chorar', 'choro', 'chorando',
    'sozinha', 'sozinho', 'estresse', 'estressado', 'estressada', 
    'dor', 'sofrer', 'sofrimento', 'agonia', 'desânimo', 'desanimo',
    'desanimada', 'desanimado', 'angustia', 'angústia', 'angustiado',
    'angustiada', 'mágoa', 'magoa', 'magoada', 'magoado', 'matar',
    'ódio', 'odeio', 'odiar', 'mal']

df_tweets_depression = pd.read_csv('clean_depression_tweets.csv')
df_users = pd.read_csv('all_users.csv')

df_tweets_depression['user_id'] = df_tweets_depression.user_id.astype(int).astype(str)
df_tweets_depression['id'] = df_tweets_depression.id.astype(int).astype(str)

In [ ]:
df_tweets_depression.drop(['user_screen_name', 'language'], axis=1).sample(5)

In [ ]:
df_tweets_depression.head()

In [ ]:
df_tweets_depression['date'] = pd.to_datetime(df_tweets_depression.date,infer_datetime_format=True)
df_tweets_depression['time'] = df_tweets_depression.date.dt.time
df_tweets_depression['hour'] = df_tweets_depression.date.dt.hour.astype(int)
# df_tweets_depression['secs'] = df_tweets_depression.time().second

In [ ]:
df_tweets_depression.dtypes

In [ ]:
df_tweets_depression.head()

### Quantidade de tweets que possuem "palavras depressivas"

In [ ]:
def is_depressive_tweet(tweet):
    for word in depression_words:
        if word in str(tweet):
            return True
    return False

In [ ]:
def count_depressive_tweets(tweets):
    tweets_with_depression_words = 0
    for tweet in tweets:
        if is_depressive_tweet(tweet):
            tweets_with_depression_words = tweets_with_depression_words + 1
    return tweets_with_depression_words
    

In [ ]:
def count_words(tweets):
    count = 0
    for tweet in tweets:
        for word in str(tweet).split():
            count = count + 1
    return count

In [ ]:
def count_depressive_words(tweets):
    count = 0
    for tweet in tweets:
        for word in str(tweet).split():
            if word in depression_words:
                count = count + 1
    return count

In [ ]:
def count_true(bool_feats):
    count = 0
    for bool_feat in bool_feats:
        if bool_feat:
            count = count + 1
    return count

In [ ]:
user_names = df_tweets_depression.user_screen_name.unique().tolist()
users = []
user = {}
for name in user_names:
    df_user_tweets = df_tweets_depression[df_tweets_depression.user_screen_name == name]
    tweets = df_user_tweets.text
    tweets_with_depression_words = count_depressive_tweets(tweets)
    words_count = count_words(tweets)
    depressive_words_count = count_depressive_words(tweets)
    has_depression = df_user_tweets.depression.unique().tolist()
    dates = df_user_tweets.date.tolist()
    rts = df_user_tweets.is_rt
    rts_count = count_true(rts)
    mentions = df_user_tweets.is_mention
    mentions_count = count_true(mentions)
    user['screen_name'] = name
    user['tweets_count'] = len(df_user_tweets)
    user['depression_tweets_count'] = tweets_with_depression_words
    user['words_count'] = words_count
    user['words_mean'] = user['words_count'] / user['tweets_count']
    user['depressive_words_count'] = depressive_words_count
    user['depression'] = has_depression[0]
    user['rts_count'] = rts_count
    user['mentions_count'] = mentions_count
    user['hour_mode'] = df_user_tweets.hour.mode().values[0]
    users.append(user.copy())

In [ ]:
df_users_metrics = pd.DataFrame(data=users)
df_users_metrics['depression_tweets_proportion'] = df_users_metrics.depression_tweets_count / df_users_metrics.tweets_count
df_users_metrics['depression_words_proportion'] = df_users_metrics.depressive_words_count / df_users_metrics.words_count

In [ ]:
df_users = df_users.drop_duplicates(['screen_name'])

In [ ]:
df_users_metrics = df_users_metrics.merge(df_users, on=['screen_name', 'depression'])

In [ ]:
df_users_metrics.head()

In [ ]:
df_users_metrics.columns

In [ ]:
df_users_metrics.drop(['depression', 'screen_name', 'id', 'name', 'location', 'created_at']
                          , axis=1).columns

In [ ]:
# df_users_metrics.to_csv('df_users_metrics.csv', index=False)